In [224]:
import pandas as pd
from langchain.chains import LLMChain, SequentialChain


In [225]:
import prompts as p


DATA_PATH = '../datasets/downsampled_50_50.csv'

SAVED_PROMPTS_PATH = '../datasets/saved_prompts.csv'


# Load data
df = pd.read_csv(DATA_PATH)

df.fillna('', inplace=True)

prompt_name = 'prompt1_cot_manual_temp_0_gpt3.5-0301'

MATCHMODEL = "gpt-3.5-turbo-0301"
EVALMODEL = "gpt-3.5-turbo-0301"


MAX_TOKENS = 15

In [226]:
# generate column with prompt_name if not exists
if 'prompt_name' not in df.columns:
    df[prompt_name] = ""

In [227]:
df.head(30)

,Unnamed: 0,id_left,brand_left,title_left,description_left,price_left,priceCurrency_left,cluster_id_left,id_right,brand_right,...,prompt_3_gpt-3.5-turbo_temp-0,prompt0_en_baseline,prompt0_en_baseline_temp_0_gpt3.5_nobrand,prompt0_en_baseline_temp_0_gpt3.5,prompt1_en_short_temp_0_gpt3.5,prompt1_en_short_temp_0_gpt3.5-0301,prompt0_en_baseline_temp_0_gpt3.5-0301,prompt1_en_short_mustafa_temp_0_gpt3.5-0301,prompt_basic_sequential_cot_temp_0_gpt3.5-0301,prompt1_cot_manual_temp_0_gpt3.5-0301
0,0,24080197,,DJI Mavic Mini Drone Fly More Combo,"Mavic Mini, the everyday flycam. Open your sky...",649.00,CAD,1709252,92817369,Fujifilm,...,False,False,False,False,False,False,False,False,False,
1,1,94788371,,Instax Mini Rainbow Film 10 Pack,Brighten the place up with this pack of rainbo...,14.99,GBP,1374090,10718715,,...,False,False,False,False,False,False,False,False,False,
2,2,14016048,,Blackmagic Design ATEM Production Studio 4K Sw...,Blackmagic Design ATEM Production Studio 4K Sw...,1.695E3,USD,1263992,52586114,,...,False,False,False,False,False,False,False,False,False,
3,3,62248973,Instax,Instax Fujifilm | Instax SQUARE Instant Film -...,,14.99,CAD,1118780,63595157,,...,False,False,False,False,False,True,False,False,False,
4,4,25011125,,Hikvision DVR 16 CH fino a 8Mpx Turbo 3.0 DS-7...,Hikvision DVR 16 CH fino a 8Mpx Turbo 3.0 DS-7...,3.726E2,EUR,1336313,10718715,,...,False,False,False,False,False,False,False,False,False,
5,5,60568221,,TAG Heuer Formula 1 Chronograph 43mm CAZ1010.F...,,1169.75,EUR,2107019,31469161,Sony,...,False,False,False,False,False,False,False,False,False,
6,6,27700752,,SanDisk Extreme PRO SD UHS-II USB 3.0,,99,PLN,137095,86534825,SanDisk,...,False,False,False,False,False,False,False,False,False,
7,7,64446755,Brother,"TN-200 sort lasertoner, original Brother TN200...",Ny original Brother sort tonerpatron til Broth...,339.00,DKK,5223239,65569829,,...,False,False,False,False,False,False,False,False,False,
8,8,54979311,,TP-Link TL-WN781ND Netcard PCI-eX Wireless 150 mb,,,,466493,34393388,,...,False,False,False,False,False,False,False,False,False,
9,9,89974024,,Refill #014: Kraft Paper Notebook,Cardboard Paper notebook refill for the Travel...,7.9,EUR,1857459,58146758,,...,False,False,False,False,False,False,False,False,False,


In [228]:
from langchain.chat_models import ChatOpenAI


In [229]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [230]:
from langfuse.callback import CallbackHandler
handler = CallbackHandler(os.environ.get("LF_PK"), os.environ.get("LF_SK"), os.environ.get("LF_HOST"))

In [231]:
from langchain.prompts import PromptTemplate


# match_prompt = PromptTemplate(
#     input_variables=["p1", "p2"],
#     template="""Your task is to decide if two product descriptions refer to the same product.

#     Think step by step!
#     Product 1: {p1}
#     Product 2: {p2}
# """
# )

match_prompt = PromptTemplate(
    input_variables=["p1", "p2"],
    template="""Q: Could you please tell me if the following two products are the same one?
Product 1: 
Title: Deore XT RD-M786-SGS - Shadow - rear derailleur
Description: · compatible with 10-speed, long cage· Dyna-Sys technology· Shimano Shadow RD+ - When the switch is in the \"on\" position, it increases the rear derailleur spring tension and reduces chain movement for a quieter and more stable ride. The \"off\" pos
Product 2: 
Title: Switch Netgear - ProSafe Switch PLUS 8porte Gigabit
Description: ProSafe Switch Plus Giga a 8 porte 10\/100\/1000Mbps -Management semplificato tramite utility...
A: These products are not the same because they are from a different brand and are used for different use cases.
Q: Could you please tell me if the following two products are the same one?
Product 1: 
Title: Daniel Wellington Classic St Mawes 40mm
Description: The Classic St. Mawes, an integral part of the flagship Classic Collection, is a slim timepiece that sits perfectly on your wrist. With a flawlessly round and simple dial, a classy leather band and an elegant casing, you have a timepiece that proves 
Product 2: 
Title: St Mawes 40mm Rose Gold Watch - Multi
Description: An integral part of the flagship classic Daniel Wellington collection, featuring a classy brown leather band a stunning rose gold case, perfectly engineered with a flawlessly round and simple dial.This watch sits perfectly on your wrist and has the a
A: These two products refer to the same real life product because they are the same Model from the same brand and have the same size.
Q: Could you please tell me if the following two products are the same one?
Product 1: 
{p1}
Product 2: 
{p2}
A:
"""
)

eval_prompt = PromptTemplate(
    input_variables=["chain_of_thought_reasoning"],
    template="""Decide if this answer corresponds to two prdocuts matching or not.
    Please answer with "match" or "no match".
    Answer: {chain_of_thought_reasoning}
"""
)




In [232]:
def cut_length(string ,length = 250):
    return string[0:length]

In [233]:

# product1 = "Title: " + cut_length(str(df["title_left"][1] or "none")) + "\nDescription: " + cut_length(str(df["description_left"][1] or "none"))
# product2 = "Title: " + cut_length(str(df["title_right"][1] or "none")) + "\nDescription: " + cut_length(str(df["description_right"][1] or "none"))



# product1 = "Title: " + cut_length(str(df["title_left"][1] or "none")) + "\nDescription: " + cut_length(str(df["description_left"][1] or "none")) + "\nBrand:" + str(df["brand_left"][1] or "none") 
# product2 = "Title: " + cut_length(str(df["title_right"][1] or "none")) + "\nDescription: " + cut_length(str(df["description_right"][1] or "none")) + "\nBrand:" + str(df["brand_right"][1] or "none") 

In [234]:
saved_prompts_df = pd.read_csv(SAVED_PROMPTS_PATH)

def saveResponse(prompt, response, isMatch, index):
    # create new column if not exists 
    if "prompt_" + prompt_name not in saved_prompts_df.columns:
        saved_prompts_df["prompt_" + prompt_name] = ""
        saved_prompts_df["response_" + prompt_name] = ""
        saved_prompts_df["isMatch_" + prompt_name] = ""


    
    # save response prompt and isMatch to df at index
    saved_prompts_df.at[index, "prompt_" + prompt_name] = prompt
    saved_prompts_df.at[index, "response_" + prompt_name] = response
    saved_prompts_df.at[index, "isMatch_" + prompt_name] = isMatch
    
    

In [235]:
def evaluate(df, temp=0, start = 0):
    match_llm = ChatOpenAI(model_name= MATCHMODEL, temperature=temp)
    eval_llm= ChatOpenAI(model_name= EVALMODEL, temperature=temp, max_tokens=MAX_TOKENS)
    for i in range(start, len(df)):
        print(i)
        # Creating Prompot and sending request and printing informations
        
        product1 = "Title: " + cut_length(str(df["title_left"][i] or "none")) + "\nDescription: " + cut_length(str(df["description_left"][i] or "none")) 
        product2 = "Title: " + cut_length(str(df["title_right"][i] or "none")) + "\nDescription: " + cut_length(str(df["description_right"][i] or "none")) 
        
        match_chain = LLMChain(llm=match_llm, prompt=match_prompt, output_key="chain_of_thought_reasoning")

        eval_chain = LLMChain(llm=eval_llm, prompt=eval_prompt, output_key="match_result")

        overall_chain = SequentialChain(chains=[match_chain, eval_chain], input_variables=["p1", "p2"], output_variables=["chain_of_thought_reasoning", "match_result"], verbose=True)
        
        try:
            response = overall_chain({"p1":product1, "p2":product2}, callbacks=[handler])
        except Exception as e:
            print(e)
            i = i - 1
            continue

        handler.langfuse.flush()

        # result = response["choices"][0]["text"]
        
        formatted_prompt = match_chain.prompt.format_prompt(p1=product1, p2=product2).text


        # if(i < 5):
        print(formatted_prompt)
        print("__\n__")
        print("chain_of_thought_reasoning: " + response["chain_of_thought_reasoning"])
        print("__\n__")
        print("match_result: " + response["match_result"])
        print("__\n__")
        
        #Processing Response
        isMatch = False
        if  "No" in response["match_result"] or "no" in response["match_result"]:
            isMatch = False
            print("no match" + str(isMatch))
        elif ("Match" in response["match_result"]) or ("match" in response["match_result"]):
            isMatch = True
            print("match" + str(isMatch))
        
        # print("isMatch", str(isMatch))
        df[prompt_name][i] = isMatch
        # saveResponse(prompt, response, isMatch)

In [238]:
evaluate(df,temp=0, start=180)


180


> Entering new SequentialChain chain...

> Finished chain.
Q: Could you please tell me if the following two products are the same one?
Product 1: 
Title: Deore XT RD-M786-SGS - Shadow - rear derailleur
Description: · compatible with 10-speed, long cage· Dyna-Sys technology· Shimano Shadow RD+ - When the switch is in the "on" position, it increases the rear derailleur spring tension and reduces chain movement for a quieter and more stable ride. The "off" pos
Product 2: 
Title: Switch Netgear - ProSafe Switch PLUS 8porte Gigabit
Description: ProSafe Switch Plus Giga a 8 porte 10\/100\/1000Mbps -Management semplificato tramite utility...
A: These products are not the same because they are from a different brand and are used for different use cases.
Q: Could you please tell me if the following two products are the same one?
Product 1: 
Title: Daniel Wellington Classic St Mawes 40mm
Description: The Classic St. Mawes, an integral part of the flagship Classic Collection, is a slim time

/var/folders/gx/tzjwcf8x1llbhhf36mjt7xqr0000gn/T/ipykernel_15256/722793016.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[prompt_name][i] = isMatch



> Finished chain.
Q: Could you please tell me if the following two products are the same one?
Product 1: 
Title: Deore XT RD-M786-SGS - Shadow - rear derailleur
Description: · compatible with 10-speed, long cage· Dyna-Sys technology· Shimano Shadow RD+ - When the switch is in the "on" position, it increases the rear derailleur spring tension and reduces chain movement for a quieter and more stable ride. The "off" pos
Product 2: 
Title: Switch Netgear - ProSafe Switch PLUS 8porte Gigabit
Description: ProSafe Switch Plus Giga a 8 porte 10\/100\/1000Mbps -Management semplificato tramite utility...
A: These products are not the same because they are from a different brand and are used for different use cases.
Q: Could you please tell me if the following two products are the same one?
Product 1: 
Title: Daniel Wellington Classic St Mawes 40mm
Description: The Classic St. Mawes, an integral part of the flagship Classic Collection, is a slim timepiece that sits perfectly on your wrist. With 

/var/folders/gx/tzjwcf8x1llbhhf36mjt7xqr0000gn/T/ipykernel_15256/722793016.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[prompt_name][i] = isMatch



> Finished chain.
Q: Could you please tell me if the following two products are the same one?
Product 1: 
Title: Deore XT RD-M786-SGS - Shadow - rear derailleur
Description: · compatible with 10-speed, long cage· Dyna-Sys technology· Shimano Shadow RD+ - When the switch is in the "on" position, it increases the rear derailleur spring tension and reduces chain movement for a quieter and more stable ride. The "off" pos
Product 2: 
Title: Switch Netgear - ProSafe Switch PLUS 8porte Gigabit
Description: ProSafe Switch Plus Giga a 8 porte 10\/100\/1000Mbps -Management semplificato tramite utility...
A: These products are not the same because they are from a different brand and are used for different use cases.
Q: Could you please tell me if the following two products are the same one?
Product 1: 
Title: Daniel Wellington Classic St Mawes 40mm
Description: The Classic St. Mawes, an integral part of the flagship Classic Collection, is a slim timepiece that sits perfectly on your wrist. With 

...

/var/folders/gx/tzjwcf8x1llbhhf36mjt7xqr0000gn/T/ipykernel_15256/722793016.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[prompt_name][i] = isMatch



> Finished chain.
Q: Could you please tell me if the following two products are the same one?
Product 1: 
Title: Deore XT RD-M786-SGS - Shadow - rear derailleur
Description: · compatible with 10-speed, long cage· Dyna-Sys technology· Shimano Shadow RD+ - When the switch is in the "on" position, it increases the rear derailleur spring tension and reduces chain movement for a quieter and more stable ride. The "off" pos
Product 2: 
Title: Switch Netgear - ProSafe Switch PLUS 8porte Gigabit
Description: ProSafe Switch Plus Giga a 8 porte 10\/100\/1000Mbps -Management semplificato tramite utility...
A: These products are not the same because they are from a different brand and are used for different use cases.
Q: Could you please tell me if the following two products are the same one?
Product 1: 
Title: Daniel Wellington Classic St Mawes 40mm
Description: The Classic St. Mawes, an integral part of the flagship Classic Collection, is a slim timepiece that sits perfectly on your wrist. With 

/var/folders/gx/tzjwcf8x1llbhhf36mjt7xqr0000gn/T/ipykernel_15256/722793016.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[prompt_name][i] = isMatch



> Finished chain.
Q: Could you please tell me if the following two products are the same one?
Product 1: 
Title: Deore XT RD-M786-SGS - Shadow - rear derailleur
Description: · compatible with 10-speed, long cage· Dyna-Sys technology· Shimano Shadow RD+ - When the switch is in the "on" position, it increases the rear derailleur spring tension and reduces chain movement for a quieter and more stable ride. The "off" pos
Product 2: 
Title: Switch Netgear - ProSafe Switch PLUS 8porte Gigabit
Description: ProSafe Switch Plus Giga a 8 porte 10\/100\/1000Mbps -Management semplificato tramite utility...
A: These products are not the same because they are from a different brand and are used for different use cases.
Q: Could you please tell me if the following two products are the same one?
Product 1: 
Title: Daniel Wellington Classic St Mawes 40mm
Description: The Classic St. Mawes, an integral part of the flagship Classic Collection, is a slim timepiece that sits perfectly on your wrist. With 

/var/folders/gx/tzjwcf8x1llbhhf36mjt7xqr0000gn/T/ipykernel_15256/722793016.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[prompt_name][i] = isMatch


In [239]:
df

,Unnamed: 0,id_left,brand_left,title_left,description_left,price_left,priceCurrency_left,cluster_id_left,id_right,brand_right,...,prompt_3_gpt-3.5-turbo_temp-0,prompt0_en_baseline,prompt0_en_baseline_temp_0_gpt3.5_nobrand,prompt0_en_baseline_temp_0_gpt3.5,prompt1_en_short_temp_0_gpt3.5,prompt1_en_short_temp_0_gpt3.5-0301,prompt0_en_baseline_temp_0_gpt3.5-0301,prompt1_en_short_mustafa_temp_0_gpt3.5-0301,prompt_basic_sequential_cot_temp_0_gpt3.5-0301,prompt1_cot_manual_temp_0_gpt3.5-0301
0,0,24080197,,DJI Mavic Mini Drone Fly More Combo,"Mavic Mini, the everyday flycam. Open your sky...",649.00,CAD,1709252,92817369,Fujifilm,...,False,False,False,False,False,False,False,False,False,False
1,1,94788371,,Instax Mini Rainbow Film 10 Pack,Brighten the place up with this pack of rainbo...,14.99,GBP,1374090,10718715,,...,False,False,False,False,False,False,False,False,False,False
2,2,14016048,,Blackmagic Design ATEM Production Studio 4K Sw...,Blackmagic Design ATEM Production Studio 4K Sw...,1.695E3,USD,1263992,52586114,,...,False,False,False,False,False,False,False,False,False,False
3,3,62248973,Instax,Instax Fujifilm | Instax SQUARE Instant Film -...,,14.99,CAD,1118780,63595157,,...,False,False,False,False,False,True,False,False,False,False
4,4,25011125,,Hikvision DVR 16 CH fino a 8Mpx Turbo 3.0 DS-7...,Hikvision DVR 16 CH fino a 8Mpx Turbo 3.0 DS-7...,3.726E2,EUR,1336313,10718715,,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216,216,71765125,TAG Heuer,TAG Heuer Formula 1 CAZ1010.FT8024,Find Authentic TAG Heuer Formula 1 CAZ1010.FT8...,1039,USD,2107019,60568221,,...,False,False,False,False,True,True,True,True,False,True
217,217,83472763,,Traveler's Refills: MD Paper - 002 / grid,"These notebooks aremade in Japan, where there ...",6.0E0,USD,2662935,66355149,,...,False,False,False,False,False,True,False,False,False,True
218,218,2251660,Traveler's Company,Traveler's Company 014. CRAFT PAPER (64PG) MID...,,4.50,GBP,1857459,89974024,,...,False,False,False,False,True,True,True,True,False,True
219,219,65569829,,MINION DHR II 29 X 2.30 3C TERRA TR DD FOLD,Refined and redesigned; ready to shred any lin...,1.0999E2,AUD,1084991,47784049,Maxxis,...,False,False,False,False,False,True,True,True,False,True


In [240]:
# write df to csv
df.to_csv(DATA_PATH, index=False)